[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/main/vizro-ai/examples/chart_by_vizro_ai.ipynb)

In [1]:
# @title
%%html
<style>
  .cell-bg {
    background-color: #E6F3FF;
    padding: 10px;
    width: 100%;
  }
  .disclaimer {
    background-color: #ffffff;
    border: 1px solid #ddd;
    border-radius: 5px;
    padding: 15px;
    margin: 10px 0;
    font-family: Arial, sans-serif;
    color: #000000;
  }
  .disclaimer h3 {
    margin-top: 0;
    color: #000000; /* Explicitly set title color for light mode */
  }
  .disclaimer ol {
    padding-left: 20px;
  }
  .disclaimer li {
    margin-bottom: 10px;
  }
  .disclaimer p {
    font-weight: bold;
    margin-bottom: 0;
  }

  /* Dark mode styles */
  @media (prefers-color-scheme: dark) {
    .cell-bg {
      background-color: #1a1a1a;
    }
    .disclaimer {
      background-color: #2a2a2a;
      border-color: #444;
      color: #ffffff;
    }
    .disclaimer h3 {
      color: #ffffff; /* Explicitly set title color for dark mode */
    }
  }
</style>

<div class="cell-bg">
  <div class="disclaimer">
    <h3>Disclaimer: Using Google Colab</h3>
    <p>Please read carefully before proceeding:</p>
    <ol>
      <li><strong>Google's Servers:</strong> This notebook runs on Google's servers. Your code and data will be processed and temporarily stored on Google's infrastructure.</li>
      <li><strong>Data Privacy:</strong> Be mindful of the data you upload or process here. Avoid using sensitive, confidential, or personal information that shouldn't be shared with third parties.</li>
      <li><strong>Security Considerations:</strong> Colab is a shared environment. While Google implements security measures, treat it as a public environment and act accordingly.</li>
      <li><strong>Terms of Service:</strong> Ensure your usage complies with Google Colab's Terms of Service and Google Cloud's Terms of Service.</li>
      <li><strong>Data Persistence:</strong> Data in Colab notebooks is not permanently stored. Save important data and results to your local machine or a secure cloud storage.</li>
      <li><strong>Resource Limitations:</strong> Colab provides free resources, but these are subject to availability and usage limits.</li>
    </ol>
    <p>By using this Colab notebook, you acknowledge that you understand and accept these conditions. Please use Colab at your own discretion and responsibility.</p>
  </div>
</div>

#### Setup the LLM API key

Secure your LLM API key as environment variable
- click the `Secrets` button on the left
- click `Add new secret`
- insert "OPENAI_API_KEY" under `Name`
- insert the API key value under `Value` (use raw string without quote. e.g., `xyz` instead of `"xyz"`)
- [optional] you might also need to insert "OPENAI_BASE_URL" and its value, based on the requirements from your API key provider (use raw string without quote. e.g., `xyz` instead of `"xyz"`)
- click the `Notebook access` toggle to make it available for the notebook run


# Use Vizro-AI to generate Vizro charts
# **At the top of this notebook, click `Runtime` -> `Run all`**

In [ ]:
import os

from google.colab import userdata

# @title ## Setup
# @markdown #### 1. Install Vizro-AI
print("Installing dependencies")
%pip install uv -q -q -q
!uv pip install --system vizro-ai --quiet
print("✅ Installation finished")

# @markdown #### 2. Setup LLM API access


def _safe_get_userdata(key):
    try:
        return userdata.get(key)
    except Exception as e:
        print(f"Warning: Unable to access {key}. Reason: {e!s}")
        return None


# Always try to set the OPENAI_API_KEY
api_key = _safe_get_userdata("OPENAI_API_KEY")
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("❌ OPENAI_API_KEY not set. Click `Secrets` icon on the left to setup.")

# Conditionally set OPENAI_BASE_URL if provided and accessible
openai_base_url = _safe_get_userdata("OPENAI_BASE_URL")
if openai_base_url:
    os.environ["OPENAI_BASE_URL"] = openai_base_url
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

print("\nCurrent environment variables:")
print(f"OPENAI_API_KEY: {'✅ Set' if 'OPENAI_API_KEY' in os.environ else 'Not set'}")
print(f"OPENAI_BASE_URL: {'✅ Set' if 'OPENAI_BASE_URL' in os.environ else 'Not set by user, use default'}")

In [ ]:
# @title ## Data upload

# @markdown #### Upload your CSV data (only 1 dataframe accepted per chart)
# @markdown - click `Choose Files` if you'd like to upload and use your own data
# @markdown - or click `Cancel upload` and use the default example data for initial exploration

import pandas as pd
from google.colab import files
from plotly.express.data import gapminder

uploaded = files.upload()

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
else:
    for fn in uploaded.keys():
        print(f'User uploaded file "{fn}"')

        df = pd.read_csv(fn)
print(f"Dataframe used for plotting: \n{df.head()}")

In [ ]:
# @title ## Input the LLM model choice and user prompt { run: "auto" }

# @markdown #### 1. Choose the LLM model

LLM = "gpt-4o"  # @param ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini"]

print(f"Selected LLM: {LLM}")

# @markdown ---
# @markdown #### 2. Input your chart question

# @markdown #### Instructions:
user_input = "Describe the composition of gdp in continent."  # @param {type:"string", placeholder:"Enter the requirements for the chart"}

print(f"User input: {user_input}")

In [ ]:
from vizro_ai import VizroAI

# @title ## Create the chart using Vizro-AI
# @markdown Note: to iterate with different data, model, and prompt, please edit in cells above and click the corresponding `Run cell` buttons.

print("Running VizroAI...")
print(f"Selected LLM: {LLM}")
print(f"User input: {user_input}")

vizro_ai = VizroAI()
result = vizro_ai.plot(df, user_input, return_elements=True)

print("✅ VizroAI finished")
print("\n-----\nCode:\n-----\n")
print(result.code)
print("\n-------\nFigure:\n-------\n")
result.get_fig_object(df).show()